In [2]:
# -*- coding: utf-8 -*-

import netCDF4 as nc4
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

In [32]:
dirt = "/gpfswork/rech/omr/ukd13rj/nemo/penalnemov4.0.1/tests/penaloverflow/exp_cfl/"
#dirm = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_CORNER/MESHMASK/"

#workdir="min_phimin0001_smo7"
#workdir="ref_zps_AimpT"
workdir="ref_zco_AimpT"

flagbpv=False

pdt = dirt + workdir + "/OVF_grid_T.nc"
pdu = dirt + workdir + "/OVF_grid_U.nc"

save = 1 ; psave = "%s.png" % (workdir)
dpi = 600 

In [33]:
dt = nc4.Dataset(pdt)
lipst   = dt.variables['lipst'  ][:,:,1,:]
cflt    = dt.variables['cflt'   ][:,:,1,:]
dt.close()
du = nc4.Dataset(pdu)
cflu    = du.variables['cflu'   ][:,:,1,:]         
lipsu   = du.variables['lipsu'  ][:,:,1,:] 
du.close()
nT,nK,nI = np.shape(cflu)

# chercher l'instant ou un des critères temporelles à frolé 1
for tb in range(nT):
    constrains = [np.max(cflu[tb]),np.max(cflt[tb]),np.max(lipsu[tb]),np.max(lipst[tb])]
    if np.max(constrains) > 1.:
        i = np.argmax(constrains)
        break

if (i==0): kk,ii = np.unravel_index(np.argmax(cflu[tb], axis=None), (nK,nI))
if (i==1): kk,ii = np.unravel_index(np.argmax(cflt[tb], axis=None), (nK,nI))
if (i==2): kk,ii = np.unravel_index(np.argmax(lipsu[tb], axis=None), (nK,nI))
if (i==3): kk,ii = np.unravel_index(np.argmax(lipst[tb], axis=None), (nK,nI))

print("cflu cflt lipsu lipst")
print(constrains)
print("timeframe %d" % tb)
print("at %d %d" % (kk,ii))

dt = nc4.Dataset(pdt)
toce    = dt.variables['thetao_inst'][tb,:,1,:]
#
lipst   = dt.variables['lipst'  ][tb,:,1,:]
cflt    = dt.variables['cflt'   ][tb,:,1,:]
cfltu   = dt.variables['cfltu'  ][tb,:,1,:]
cfltw   = dt.variables['cfltw'  ][tb,:,1,:]

if flagbpv : 
    rpot    = dt.variables['rpot'   ][tb,:,1,:]
    rlipst  = dt.variables['rlipst' ][tb,:,1,:]
    rphiu_t = dt.variables['rphiu_t'][tb,:,1,:]
    rphiw_t = dt.variables['rphiw_t'][tb,:,1,:]
#
dt.close()

du = nc4.Dataset(pdu)
#
cflu    = du.variables['cflu'   ][tb,:,1,:]      
cfluu   = du.variables['cfluu'  ][tb,:,1,:]    
cfluw   = du.variables['cfluw'  ][tb,:,1,:]    
lipsu   = du.variables['lipsu'  ][tb,:,1,:]    
if flagbpv : 
    rlipsu  = du.variables['rlipsu' ][tb,:,1,:] 
    rphiu_u = du.variables['rphiu_u'][tb,:,1,:]
    rphiw_u = du.variables['rphiw_u'][tb,:,1,:]
#
du.close()
print("data loaded")

cflu cflt lipsu lipst
[0.2895081, 0.3111444, 0.008290825, 0.015954122]
timeframe 101
at 39 97
data loaded


In [34]:
palette = plt.get_cmap('RdBu_r',10)
optpcolor = {"cmap" : palette}
optptoce = {"cmap" : palette, "vmin" : 10, "vmax" : 20}

NI = 3 ; NJ = 5 # nb of rows (NI) and cols (NJ)
fig, ax = plt.subplots(NI,NJ, figsize=(NJ*6,NI*4), dpi = dpi, squeeze=False)
# settled per rows (i=0) j=0 j=1 j=2
#                  (i=1) j=0 j=1 j=2
im = [[],[],[]]

""" 1st row : toce lipsu rlipsu lipst rlipst
"""
im[0].append( ax[0][0].imshow(toce     , **optptoce)   )  ; ax[0][0].set_title('%s (%d)' % (workdir,tb))
im[0].append( ax[0][1].imshow(lipsu    , **optpcolor)  )  ; ax[0][1].set_title(' lipsu max %.1f' % ( lipsu[kk,ii]) )
im[0].append( ax[0][2].imshow(rlipsu   , **optpcolor)  )  ; ax[0][2].set_title('rlipsu max %.1f' % ( rlipsu[kk,ii]) )
im[0].append( ax[0][3].imshow(lipst    , **optpcolor)  )  ; ax[0][3].set_title(' lipst max %.1f' % ( lipst[kk,ii]) )
im[0].append( ax[0][4].imshow(rlipst   , **optpcolor)  )  ; ax[0][4].set_title('rlipst max %.1f' % ( rlipst[kk,ii]) )
""" 2nd row : cflu cfluu rphiu_u cfluw rphiw_u
"""
im[1].append( ax[1][0].imshow(cflu     , **optpcolor) )  ; ax[1][0].set_title('   cflu max %.1f' % ( cflu[kk,ii]) )
im[1].append( ax[1][1].imshow(cfluu    , **optpcolor) )  ; ax[1][1].set_title('  cfluu max %.1f' % ( cfluu[kk,ii]) )
im[1].append( ax[1][2].imshow(rphiu_u  , **optpcolor) )  ; ax[1][2].set_title('rphiu_u max %.1f' % ( rphiu_u[kk,ii]) )
im[1].append( ax[1][3].imshow(cfluw    , **optpcolor) )  ; ax[1][3].set_title('  cfluw max %.1f' % ( cfluw[kk,ii]) )
im[1].append( ax[1][4].imshow(rphiw_u  , **optpcolor) )  ; ax[1][4].set_title('rphiw_u max %.1f' % ( rphiw_u[kk,ii]) )

""" 3rd row : cflt cfltu rphiu_t cfltw rphiw_t
"""
im[2].append( ax[2][0].imshow(cflt     , **optpcolor) ) ; ax[2][0].set_title('   cflt max %.1f' % ( cflt[kk,ii]) )
im[2].append( ax[2][1].imshow(cfltu    , **optpcolor) ) ; ax[2][1].set_title('  cfltu max %.1f' % ( cfltu[kk,ii]) )
im[2].append( ax[2][2].imshow(rphiu_t  , **optpcolor) ) ; ax[2][2].set_title('rphiu_t max %.1f' % ( rphiu_t[kk,ii]) )
im[2].append( ax[2][3].imshow(cfltw    , **optpcolor) ) ; ax[2][3].set_title('  cfltw max %.1f' % ( cfltw[kk,ii]) )
im[2].append( ax[2][4].imshow(rphiw_t  , **optpcolor) ) ; ax[2][4].set_title('rphiw_t max %.1f' % ( rphiw_t[kk,ii]) )


if flagbvp :
    ax[0][2].axis('off')
    ax[0][3].axis('off')
    ax[0][2].axis('off')
    ax[0][2].axis('off')
    ax[0][2].axis('off')
    ax[0][2].axis('off')
""" pretty
"""
#fig.colorbar(cf, ax=ax[i:(i+1)][j:(j+1)], shrink=0.8, location='bottom'),
#                 aspect = 60, fraction=0.05,  extend = 'both', pad = 0.02)
for i in range(NI):
    for j in range(NJ):
        ax[i][j].set_xticklabels([]) ; ax[i][j].set_yticklabels([])
        ax[i][j].set_xticks([])        ; ax[i][j].set_yticks([]) 
        ax[i][j].set_aspect(aspect='equal')
        fig.colorbar(im[i][j],ax=ax[i:(i+1),j:(j+1)], shrink=0.8, location='bottom',
                    pad = 0.02)
        
fig.patch.set_facecolor('white')
#plt.subplots_adjust(wspace=0.04, hspace=0.08)
if save :
    print("\nsaving : %s" % psave)
    fig.savefig(psave, dpi = dpi)
    plt.close()
    print("figure closed")
plt.show()


saving : ref_zco_AimpT.png
figure closed
